Genera una planilla de excel con los resultados de lactura de datos desde una base de datos y envío del resultado como archivo vía correo.

In [16]:
'''
 vista_entregas_cloud, enviando ese archivo mediante correo electronico
Autor: Patricio Araneda
Fecha: 2023-12-31
'''
#librerias
import os   
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine, text, select
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from openpyxl import Workbook


In [38]:
# Azure connection
server = os.environ["sql_host"]
database = os.environ["sql_db"] 
username = os.environ["sql_user"]
password = os.environ["sql_pwd"]

conn_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';AUTOCOMMIT=FALSE;UID='+username+';PWD='+ password
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}", fast_executemany=True)

# archivo resultante
filename = 'entregas.xlsx'

In [ ]:
# seleccion de datos de la base de datos.
sqldf = pd.read_sql(text("select * from vistadedatos"), engine)
data = pd.DataFrame(sqldf)
data.head()

In [40]:
# sen email with excel file

# Convert DataFrame to Excel
data.to_excel(filename, index=False)


In [41]:
# Create email message
msg = MIMEMultipart()

msg['From'] = 'paranedagarcia@gmail.com'
msg['To'] = 'paranedagarcia@gmail.com, patricio@aranedagarcia.cl'
msg['Subject'] = 'Unidades a entregar'

# Attach the Excel file
part = MIMEBase('application', "octet-stream")
part.set_payload(open(filename, "rb").read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', 'attachment', filename=filename)  # or whatever the filename you want to send
msg.attach(part)

# Send the email
smtp = smtplib.SMTP('smtp.gmail.com', 587)
smtp.starttls()
smtp.login('paranedagarcia@gmail.com', 'clavecorreo')
smtp.send_message(msg)
smtp.quit()

(221,
 b'2.0.0 closing connection d21-20020a17090abf9500b0028c2b2b76c0sm15482000pjs.22 - gsmtp')